In [5]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def job():
    header = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.89 Safari/537.36"  
    }

    listagem = []
    url = 'https://www.ivalor.com.br/empresas/listagem?p=1'
    
    for page_number in range(1, 29):
        urlbase = url.replace('p=1', 'p=' + str(page_number))

        response = requests.get(urlbase, headers=header)
        site = BeautifulSoup(response.text, 'html.parser')
        tabela = site.find('tbody', id='tab_empresas_body')

        if tabela:
            rows = tabela.find_all('tr')
            for row in rows:
                cells = row.find_all('td')
                if len(cells) >= 5:  # Garantir que temos células suficientes
                    Index = cells[0].text.strip()
                    Empresa = cells[1].text.strip()
                    acoes = cells[2].find_all('a')
                    Classificacao = cells[3].text.strip()
                    Setor = cells[4].text.strip()

                    # Expandir os papéis, criando uma nova linha para cada papel
                    for acao in acoes:
                        Papél = acao.text.strip()
                        listagem.append([Index, Empresa, Papél, Classificacao, Setor])

    # Criar o DataFrame
    df = pd.DataFrame(listagem, columns=['Index', 'Empresa', 'Papél', 'Classificacao', 'Setor'])
    
    return df

# Executa a função e obtém o DataFrame
df_resultado = job()

# Mostra as primeiras linhas do DataFrame
print(df_resultado.head())

# Exporta o DataFrame para um arquivo Excel na pasta atual da aplicação
output_file = "listagem_empresas.xlsx"
df_resultado.to_excel(output_file, index=False)
print(f"Arquivo Excel exportado para: {output_file}")


  Index       Empresa  Papél Classificacao                              Setor
0     1  TRAN PAULIST  TRPL3       Mid Cap  Nível 1 de Governança Corporativa
1     1  TRAN PAULIST  TRPL4       Mid Cap  Nível 1 de Governança Corporativa
2     2         COPEL  CPLE3       Big Cap  Nível 2 de Governança Corporativa
3     2         COPEL  CPLE5       Big Cap  Nível 2 de Governança Corporativa
4     2         COPEL  CPLE6       Big Cap  Nível 2 de Governança Corporativa
Arquivo Excel exportado para: listagem_empresas.xlsx


In [1]:
import requests
import pandas as pd

def fetch_data(mesAnoInicio, mesAnoFim, num_pages, api_key):
    """
    Puxa os dados da API de despesas de recursos recebidos e retorna um DataFrame.

    Args:
    - mesAnoInicio (str): Data de início no formato MM/YYYY.
    - mesAnoFim (str): Data de fim no formato MM/YYYY.
    - num_pages (int): Número de páginas a serem puxadas.
    - api_key (str): Chave da API para autenticação.

    Returns:
    - DataFrame: Dados retornados pela API.
    """
    # URL da API
    url = 'https://api.portaldatransparencia.gov.br/api-de-dados/despesas/recursos-recebidos'

    # Parâmetros iniciais da requisição
    params = {
        'mesAnoInicio': mesAnoInicio,
        'mesAnoFim': mesAnoFim,
        'pagina': '1'
    }

    # Cabeçalhos da requisição
    headers = {
        'accept': '*/*',
        'chave-api-dados': 'e4454267ff870f9340c5b8b21b68d5ef'
    }

    # Lista para armazenar os dados
    all_data = []

    # Iterar sobre as páginas
    for _ in range(num_pages):
        # Fazendo a requisição GET
        response = requests.get(url, headers=headers, params=params)
        
        # Verificando o status da resposta
        if response.status_code == 200:
            # Convertendo a resposta para JSON
            data = response.json()
            
            # Verificando se a resposta está vazia (sem mais dados)
            if not data:
                break
            
            # Adicionando os dados à lista
            all_data.extend(data)
            
            # Incrementando o número da página para a próxima iteração
            params['pagina'] = str(int(params['pagina']) + 1)
        else:
            print(f'Erro na requisição: {response.status_code}')
            break

    # Convertendo a lista de dados para um DataFrame do pandas
    df = pd.DataFrame(all_data)

    return df

# Exemplo de uso
mesAnoInicio = '04/2024'
mesAnoFim = '05/2024'
num_pages = 100  # Número de páginas que você quer puxar
api_key = 'e4454267ff870f9340c5b8b21b68d5ef'

# Chamando a função e obtendo o DataFrame
df = fetch_data(mesAnoInicio, mesAnoFim, num_pages, api_key)

# Mostrando os dados no DataFrame
df.head()



,anoMes,codigoPessoa,nomePessoa,tipoPessoa,municipioPessoa,siglaUFPessoa,codigoUG,nomeUG,codigoOrgao,nomeOrgao,codigoOrgaoSuperior,nomeOrgaoSuperior,valor
0,202404,***.622.652-**,MARIA VICTORIA DA COSTA DOS REIS,Pessoa Física,CAPITÃO POÇO,PA,153034,UNIVERSIDADE FEDERAL RURAL DA AMAZONIA,26253,Universidade Federal Rural da Amazônia,26000,Ministério da Educação,470.00
1,202404,***.128.532-**,RAYAN DAVID SANTOS DA SILVA,Pessoa Física,BELÉM,PA,153034,UNIVERSIDADE FEDERAL RURAL DA AMAZONIA,26253,Universidade Federal Rural da Amazônia,26000,Ministério da Educação,553.96
2,202404,00.360.305/0001-04,CAIXA ECONOMICA FEDERAL,Entidades Empresariais Privadas,BRASÍLIA,DF,154421,FUND. UNIVERSIDADE FEDERAL VALE SAO FRANCISCO,26230,Fundação Universidade Federal do Vale do São F...,26000,Ministério da Educação,2751130.16
3,202404,EX87CRIST,CRISTAL SPRINGS,Administração Pública,Exterior,EX,240087,EMBAIXADA DO BRASIL EM WASHINGTON,35000,Ministério das Relações Exteriores - Unidades ...,35000,Ministério das Relações Exteriores,4822.06
4,202404,09.112.236/0001-94,FUNDACAO NAPOLEAO LAUREANO,Entidades Sem Fins Lucrativos,JOÃO PESSOA,PB,783330,CAPITANIA DOS PORTOS DA PARAIBA,52131,Comando da Marinha,52000,Ministério da Defesa,4533.88


In [3]:
import requests
import pandas as pd

# Chave da API
api_key = '2A7SLT26JL4H11LS'
symbol = 'IBM'

# Fazer a requisição para a API Alpha Vantage
url = f'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={symbol}&apikey={api_key}'
response = requests.get(url)
data = response.json()

# Extrair a série temporal dos dados diários
time_series = data['Time Series (Daily)']

# Converter em DataFrame
df = pd.DataFrame.from_dict(time_series, orient='index')

# Renomear colunas
df.columns = ['Open', 'High', 'Low', 'Close', 'Volume']

# Converter índices para o tipo datetime
df.index = pd.to_datetime(df.index)

# Exibir as primeiras linhas do DataFrame
print(df.head())


                Open      High       Low     Close   Volume
2024-09-23  218.0000  220.6200  217.2700  220.5000  3928369
2024-09-20  214.3300  217.8500  213.7400  217.7000  9958980
2024-09-19  218.0100  218.4800  210.3700  213.8900  5279559
2024-09-18  214.1300  216.8600  213.5900  214.9400  3482764
2024-09-17  217.2500  218.8400  213.0000  214.1300  5635210
